In [ ]:
import requests
import json
import pandas as pd
from time import sleep

# Função para ajustar o valor para 14 caracteres
def pad_zeros(value):
    value_str = str(value)  # Converter o valor para string
    return value_str.zfill(14) if len(value_str) < 14 else value_str

# Ler a planilha
#dados_cnpj = pd.read_excel('CNPJS sem municipio.xlsx')
dados_cnpj = pd.read_excel('CNPJs.xlsx')

# Aplicar a função para ajustar os valores
dados_cnpj['coluna_ajustada'] = dados_cnpj['CNPJ'].apply(pad_zeros)
lista_cnpj = dados_cnpj['coluna_ajustada'].tolist()

# Função para consultar o CNPJ na API
def consulta_cnpj(cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{cnpj}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Erro na requisição para o CNPJ {cnpj}. Código de status: {response.status_code}")
        return None
    
    try:
        resp = json.loads(response.text)
    except json.JSONDecodeError:
        print(f"Erro ao decodificar o JSON para o CNPJ {cnpj}. Resposta: {response.text}")
        return None
    
    try:
        return {
            "Municipio": resp["municipio"],
            "UF": resp["uf"],
            "Logradouro": resp["logradouro"],
            "Numero": resp["numero"],
            "CNPJ": resp["cnpj"],
            "Situação": resp["situacao"],
            "CEP": resp["cep"],
            "Bairro": resp["bairro"],
            "Complemento": resp["complemento"],
            "Nome Fantasia": resp["fantasia"],
            "Razão Social": resp["nome"]
        }
    except KeyError as e:
        print(f"Chave ausente no JSON para o CNPJ {cnpj}: {e}")
        return None

# Lista para armazenar os resultados
resultados = []

# Loop para consultar cada CNPJ
for i in range(200, 223): #len(lista_cnpj)
    resultado = consulta_cnpj(lista_cnpj[i])
    
    if resultado is not None:
        resultados.append(resultado)
    else:
        print(f"Erro ao consultar o CNPJ {lista_cnpj[i]}. Nenhum dado retornado.")
    
    sleep(21)  # Pausa de 15 segundos entre as requisições

# Convertendo a lista de dicionários para um DataFrame do Pandas
data = pd.DataFrame(resultados)

# Salvar os resultados em um novo arquivo Excel
data.to_excel('resultados_cnpj3.xlsx', index=False)